# Orders API with Planet SDK 

Getting started with the Orders API using the Planet SDK.
This tutorial uses [Planet](https://www.planet.com)'s Orders API with the official [Python client](https://github.com/planetlabs/planet-client-python).

## Requirements

An account on the [Planet Platform](https://www.planet.com/account/) is required to access any of Planet's API's. If you are not logged in, you will be prompted to do so below in the *SDK Authentication* section.

## Useful links 
* [Planet SDK for Python](https://planet-sdk-for-python.readthedocs.io/en/stable/get-started/quick-start-guide/)
* [Planet Python Client Repo](https://github.com/planetlabs/planet-client-python)
* [Planet Orders API Documentation](https://docs.planet.com/develop/apis/orders/)

___

# Set up

In order to interact with the Planet API using the Python client, we need to import the necessary packages and authenticate our Planet account credentials.

In [20]:
import os
from planet import Auth, Planet, Session, order_request, reporting

Your Planet login is used to authenticate and activate the Python SDK.
If you are not currently logged in to your Planet account, you will be prompted via a login link to do so. Confirm on the page that the code displayed matches the authorization code below. 
If you would like to know more, please visit the [authentication documentation](https://docs.planet.com/develop/authentication).

In [ ]:
# OAuth2 python client authentication
# If you are not already logged in, this will prompt you to open a web browser to log in.

auth = Auth.from_profile('planet-user', save_state_to_storage=True)
auth.ensure_initialized(allow_open_browser=False, allow_tty_prompt=True)

session = Session(auth)
pl = Planet(session)

## Building an Order request

#### Manual JSON Order requests

You can build an Order request manually by using a blob of JSON -- you may have used this with Python Client V1, or with `requests` directly to the API:

In [ ]:
# No need to write this all out with the SDK!

request = {
  "name": "test_order_sdk_method_1",
  "products": [
    {
     "item_ids":[
        "20200922_183724_23_106a",
        "20200922_183722_17_106a"
         ],
      "item_type": "PSScene",
      "product_bundle": "analytic_udm2"
    }
  ],
   "tools": [
    {
      "reproject": {
        "projection": "EPSG:4326",
        "kernel": "cubic"
      }
    }
  ]
}

### Orders with Planet SDK

In SDK V2 and newer, you can build an Order request block using `build_request`. You can specify all Order details including products, bundles/fallback bundles, optional cloud delivery configuration, tools & toolchain operations, etc. [Read more in the docs here](https://docs.planet.com/develop/apis/orders/).

#### Build the Order Request

Here we will build an order request. We will define for this example:
- Products, with 2 `item_ids`,  with the `product_bundle` of 'analytic_udm2', and the `item_type` of 'PSScene'
- Optional `tools`, which in this case is the `reproject_tool` 
    - `order_request.clip_tool` could also be used here and added to the `tools` list.
    - This would require a GeoJson aoi.

In [15]:
order_request.product?

Object `order_request.product` not found.


In [22]:
# Define request details to properly build your request

item_ids = ["20200922_183724_23_106a", "20200922_183722_17_106a"]

products = [
    order_request.product(item_ids, 'analytic_udm2', 'PSScene')
]

tools = [
    order_request.reproject_tool(projection='EPSG:4326', kernel='cubic')
]

request = order_request.build_request(
    'test_order_from_sdk', products=products, tools=tools)


In [23]:
# View your built order_request:
request

{'name': 'test_order_from_sdk',
 'products': [{'item_ids': ['20200922_183724_23_106a',
    '20200922_183722_17_106a'],
   'item_type': 'PSScene',
   'product_bundle': 'analytic_udm2'}],
 'tools': [{'reproject': {'projection': 'EPSG:4326', 'kernel': 'cubic'}}]}

### Create the Order

The next step after building an Order is to send it to the Orders API to create it using 'create_order'.

To do this, we'll create a `Session` to manage our communcation with Planet in general -- this will make use of that `auth` object we created earlier. Within the context of this Session, we'll create an Orders API-specific `client` to handle interactions with the Orders API.

After the order is created, we can view the returned dictionary of created_order, which will contain the new `order_id`.

In [24]:
created_order = pl.orders.create_order(request)

Congratulations! You have created an order. You can view it below:

In [25]:
created_order

{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/18389c7e-1831-4de7-8187-d30ffe88b4b7'},
 'created_on': '2025-10-01T20:41:21.52333Z',
 'error_hints': [],
 'id': '18389c7e-1831-4de7-8187-d30ffe88b4b7',
 'last_message': 'Preparing order',
 'last_modified': '2025-10-01T20:41:21.52333Z',
 'name': 'test_order_from_sdk',
 'products': [{'item_ids': ['20200922_183724_23_106a',
    '20200922_183722_17_106a'],
   'item_type': 'PSScene',
   'product_bundle': 'analytic_udm2'}],
 'state': 'queued',
 'tools': [{'reproject': {'kernel': 'cubic', 'projection': 'EPSG:4326'}}]}

______

### Get your Order after it is created

When you first submit your order, the `state` key in the order you created is 'queued'. Our variable *created_order* is just the json return of the order creation, and will not be updated automatically when the order is `running` or `success`. Instead, use the methods below that utilize `get_order` from the orders API to get our order, and its download links. More information about order states may be found [here](https://docs.planet.com/develop/apis/orders/#states).

#### Manually get the Order

In [29]:
# We can run get_order() until the status is 'complete'...
order_id = created_order['id']

order_output = pl.orders.get_order(order_id)
print(f"Your order request state is: {order_output['state']}")

Your order request state is: success


In [ ]:
# The link to download the order components is is the 'location' key of 'results'
order_output

#### Alternative: Monitor Order status until complete, then download the Order

Instead of manually checking our order's state with `orders.get_order`, we can instead use `orders.wait()` with  `reporting.StateBar()` that will run until our order is ready. The order will be downloaded with `download_order` to the `output` directory afterwords.

In [31]:
# You may need to wait a few minutes for the order to complete
order_id = created_order['id']
order = pl.orders.get_order(order_id)
print(f"Your order request state is: {order_output['state']}")
with reporting.StateBar() as bar:
    pl.orders.wait(order_id, callback=bar.update_state)

# if we get here that means the order completed. Yay! Download the files.
cwd = os.getcwd()
output_dir = os.path.join(cwd, 'output')
output_paths = pl.orders.download_order(order_id, output_dir)
output_paths = [os.path.abspath(path) for path in output_paths]
output_paths

Your order request state is: success


00:00 - order  - state: success


['/home/jovyan/work/jupyter-notebooks/Orders-API/output/18389c7e-1831-4de7-8187-d30ffe88b4b7/PSScene/20200922_183722_17_106a_3B_AnalyticMS_reproject.tif',
 '/home/jovyan/work/jupyter-notebooks/Orders-API/output/18389c7e-1831-4de7-8187-d30ffe88b4b7/PSScene/20200922_183722_17_106a_metadata.json',
 '/home/jovyan/work/jupyter-notebooks/Orders-API/output/18389c7e-1831-4de7-8187-d30ffe88b4b7/PSScene/20200922_183722_17_106a_3B_udm2_reproject.tif',
 '/home/jovyan/work/jupyter-notebooks/Orders-API/output/18389c7e-1831-4de7-8187-d30ffe88b4b7/PSScene/20200922_183722_17_106a_3B_AnalyticMS_metadata.xml',
 '/home/jovyan/work/jupyter-notebooks/Orders-API/output/18389c7e-1831-4de7-8187-d30ffe88b4b7/PSScene/20200922_183724_23_106a_metadata.json',
 '/home/jovyan/work/jupyter-notebooks/Orders-API/output/18389c7e-1831-4de7-8187-d30ffe88b4b7/PSScene/20200922_183724_23_106a_3B_udm2_reproject.tif',
 '/home/jovyan/work/jupyter-notebooks/Orders-API/output/18389c7e-1831-4de7-8187-d30ffe88b4b7/PSScene/20200922_1